### Import libraries

### Connecting to DB


In [22]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db_connection import start_engine
import pandas as pd
engine = start_engine()

### Function for pivot table

In [25]:
def file_transforming(file, column_name):
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df = df[(df["Jahr"] == "Januar")| (df["Jahr"] == "Februar")| 
        (df["Jahr"] == "März")| (df["Jahr"] == "April")| 
        (df["Jahr"] == "Mai")| (df["Jahr"] == "Juni")| 
        (df["Jahr"] == "Juli")| (df["Jahr"] == "August")| 
        (df["Jahr"] == "September")| (df["Jahr"] == "Oktober")| 
        (df["Jahr"] == "November")| (df["Jahr"] == "Dezember")] # Using only total year data
    df_long = df.melt( # Make a row as a column
        id_vars = ["Jahr"],
        var_name = "year",
        value_name = column_name
    )
    df_long.rename(columns={"Jahr": "month"}, inplace=True)
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce") # Get rid of mistakes with numbers 
    df_long = df_long.dropna(subset=[column_name]) # Drop info with N/A data
    df_final = df_long[df_long["year"] <= 2024] 
    df_final = df_final[["year","month", column_name]].reset_index(drop=True) # Choosing final data to show 
    return df_final.to_sql(name = column_name, con = engine, # Push to DB
                           if_exists= "replace", index = False)



In [26]:
# Files name
electricity_consumptions = "monatszahlen_energie_und_wasserversorgung_abgabe_an_letztverbraucher_stromverbrauch_25-11-2025.csv"
electricity_generation = "monatszahlen_energie_und_wasserversorgung_elektrizitaetserzeugung_brutto_25-11-2025.csv"
renewable_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_erneuerbare_energien_25-11-2025.csv"
fossil_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_fossile_und_andere_energietraeger_25-11-2025.csv"
total_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_insgesamt_25-11-2025.csv"
# Running functions to push 
consumed_energie_GWh = file_transforming(electricity_consumptions, "monthly_consumed_energie_GWh")
file_transforming(electricity_generation, "monthly_generated_energie_GWh")
file_transforming(renewable_feed_in, "monthly_renewable_feed_in_GWh")
file_transforming(fossil_feed_in, "monthly_fossil_feed_in_GWh")
file_transforming(total_feed_in, "monthly_total_feed_in_GWh")

84

In [15]:
import pandas as pd

df = pd.read_csv("monatszahlen_energie_und_wasserversorgung_abgabe_an_letztverbraucher_stromverbrauch_25-11-2025.csv", encoding="latin1", sep=";")
df = df[(df["Jahr"] == "Insgesamt") | (df["Jahr"] == "Januar")| (df["Jahr"] == "Februar")| 
        (df["Jahr"] == "März")| (df["Jahr"] == "April")| 
        (df["Jahr"] == "Mai")| (df["Jahr"] == "Juni")| 
        (df["Jahr"] == "Juli")| (df["Jahr"] == "August")| 
        (df["Jahr"] == "September")| (df["Jahr"] == "Oktober")| 
        (df["Jahr"] == "November")| (df["Jahr"] == "Dezember")]
df_long = df.melt( # Make a row as a column
    id_vars = ["Jahr"],
    var_name = "year",
    value_name = "consumed_energie_GWh")
df_long.rename(columns={"Jahr": "month"}, inplace=True)
df_long

,month,year,consumed_energie_GWh
0,Januar,1999,NaN
1,Februar,1999,NaN
2,März,1999,NaN
3,April,1999,NaN
4,Mai,1999,NaN
...,...,...,...
359,September,Unnamed: 28,NaN
360,Oktober,Unnamed: 28,NaN
361,November,Unnamed: 28,NaN
362,Dezember,Unnamed: 28,NaN
